In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("data/2008.csv")

In [3]:
df.head(100)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003,1955,2211,2225,WN,335,...,4,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754,735,1002,1000,WN,3231,...,5,10,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628,620,804,750,WN,448,...,3,17,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926,930,1054,1100,WN,1746,...,3,7,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829,1755,1959,1925,WN,3920,...,3,10,0,NaN,0,2,0,0,0,32
5,2008,1,3,4,1940,1915,2121,2110,WN,378,...,4,10,0,NaN,0,NaN,NaN,NaN,NaN,NaN
6,2008,1,3,4,1937,1830,2037,1940,WN,509,...,3,7,0,NaN,0,10,0,0,0,47
7,2008,1,3,4,1039,1040,1132,1150,WN,535,...,7,7,0,NaN,0,NaN,NaN,NaN,NaN,NaN
8,2008,1,3,4,617,615,652,650,WN,11,...,6,19,0,NaN,0,NaN,NaN,NaN,NaN,NaN
9,2008,1,3,4,1620,1620,1639,1655,WN,810,...,3,6,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [12]:
delays = df.groupby(['UniqueCarrier'])['DepDelay', 'ArrDelay', 
                                       'LateAircraftDelay', 
                                       'SecurityDelay', 
                                       'NASDelay',
                                       'WeatherDelay',
                                       'CarrierDelay'].mean()

delays.to_csv('data/2008delays.csv')

delays

,DepDelay,ArrDelay,LateAircraftDelay,SecurityDelay,NASDelay,WeatherDelay,CarrierDelay
UniqueCarrier,,,,,,,
9E,6.765860,4.111135,19.808549,0.033956,14.492992,3.263802,19.864633
AA,13.280898,12.607194,20.454338,0.040644,18.324412,2.568192,17.309764
AQ,-1.397783,-2.888674,13.624365,0.269036,2.510152,1.494924,22.149746
AS,6.848722,4.804346,19.587380,0.234713,12.570388,1.571083,15.960020
B6,12.653396,11.084184,26.272982,0.042683,26.604270,1.121874,13.766349
CO,13.185230,10.979037,17.169932,0.155954,26.250318,2.741824,13.637261
DL,8.007766,7.855163,16.189377,0.013810,19.680231,1.690854,14.277823
EV,11.922538,10.208002,9.813471,0.053472,16.946573,8.492954,25.375898
F9,5.919602,6.108247,9.488708,0.019799,18.044055,1.082135,12.708679


In [5]:
df.to_csv('data/2008.tsv', sep='\t')

In [6]:
df['DayofMonth'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  2,  1])

In [7]:
delays_airport = df.groupby(['Origin'])['DepDelay', 'ArrDelay', 
                                       'LateAircraftDelay', 
                                       'SecurityDelay', 
                                       'NASDelay',
                                       'WeatherDelay',
                                       'CarrierDelay'].mean()

delays_airport.fillna(0)

delays_airport = delays_airport.sort(['DepDelay'], ascending=False)

delays_airport.head(1000)

,DepDelay,ArrDelay,LateAircraftDelay,SecurityDelay,NASDelay,WeatherDelay,CarrierDelay
Origin,,,,,,,
ACK,29.854415,28.603819,24.432099,0.000000,32.993827,7.567901,19.228395
PUB,27.000000,74.000000,0.000000,0.000000,47.000000,27.000000,0.000000
CEC,24.186207,24.670949,23.938303,0.000000,41.226221,2.017995,11.123393
PIR,22.800000,23.800000,0.000000,0.000000,19.000000,0.000000,99.000000
SPI,22.314516,21.950673,24.331361,0.236686,49.000000,0.742604,19.899408
AKN,21.938596,22.044643,28.132075,1.660377,3.056604,0.000000,12.849057
OTH,21.765531,26.381048,30.106280,0.033816,29.811594,0.792271,3.603865
LMT,20.046559,20.323770,24.181818,0.000000,37.707792,1.798701,15.818182
ACY,18.548673,5.410714,0.000000,0.000000,66.428571,0.000000,75.928571


In [10]:
delays_airport.to_csv('data/2008delays_air.tsv', sep = "\t")

# Data for Chord

In [29]:
import pandasql

delays_sql = """
    select Origin, 
           Dest, 
           (avg(DepDelay) + avg(ArrDelay) + avg(LateAircraftDelay) + 
            avg(NASDelay) + avg(WeatherDelay) + avg(CarrierDelay)) as delays
    from df
    group by Origin, Dest
    """

orig_dest_delay = pandasql.sqldf(delays_sql.lower(), locals())

orig_dest_delay['delays'] = orig_dest_delay['delays'].fillna(0)

#orig_dest_delay = orig_dest_delay.sort(['delays'], ascending=0)
orig_dest_delay.head()

,Origin,Dest,delays
0,ABE,ATL,91.610286
1,ABE,BHM,0.000000
2,ABE,CLE,42.970444
3,ABE,CLT,35.327595
4,ABE,CVG,38.452412


In [101]:
#https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States
#airports = ['ATL', 'LAX', 'ORD', 'DFW', 'JFK', 'DEN', 'SFO',
#            'CLT', 'LAS', 'PHX', 'IAH', 'MIA', 'SEA', 'EWR',
#            'MCO', 'MSP', 'DTW', 'BOS', 'PHL', 'LGA']

airports = df['Origin'].unique()
airports

array(['IAD', 'IND', 'ISP', 'JAN', 'JAX', 'LAS', 'LAX', 'LBB', 'LIT',
       'MAF', 'MCI', 'MCO', 'MDW', 'MHT', 'MSY', 'OAK', 'OKC', 'OMA',
       'ONT', 'ORF', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'PVD', 'RDU',
       'RNO', 'RSW', 'SAN', 'SAT', 'SDF', 'SEA', 'SFO', 'SJC', 'SLC',
       'SMF', 'SNA', 'STL', 'TPA', 'TUL', 'TUS', 'ABQ', 'ALB', 'AMA',
       'AUS', 'BDL', 'BHM', 'BNA', 'BOI', 'BUF', 'BUR', 'BWI', 'CLE',
       'CMH', 'CRP', 'DAL', 'DEN', 'DTW', 'ELP', 'FLL', 'GEG', 'HOU',
       'HRL', 'ROC', 'DAY', 'ORD', 'EWR', 'SYR', 'IAH', 'LFT', 'MKE',
       'CHS', 'LCH', 'CLT', 'BTR', 'CRW', 'FAT', 'COS', 'MRY', 'LGB',
       'BFL', 'EUG', 'ICT', 'MEM', 'LGA', 'DCA', 'BTV', 'GRK', 'BRO',
       'TYS', 'DSM', 'BPT', 'GPT', 'GRR', 'PWM', 'MSP', 'RIC', 'CVG',
       'SAV', 'SRQ', 'GSO', 'CHA', 'XNA', 'GSP', 'LEX', 'MFE', 'ABE',
       'MLU', 'MOB', 'LRD', 'SHV', 'TLH', 'CAE', 'AEX', 'ATL', 'DFW',
       'BGR', 'AVL', 'BOS', 'MSN', 'HSV', 'MGM', 'MYR', 'VPS', 'CLL',
       'PNS', 'MTJ',

In [102]:
import random
import pprint

airports = map(lambda x: (x, "#%06x" % random.randint(0, 0xFFFFFF)), airports)

with open('airliners.csv', 'w') as a:
    a.write("name,color\n")
    count = 0
    for i in airports:
        a.write("{}, {}\n".format(i[0], i[1]))
        count += 1
        
len(airports)

303

In [103]:
from pandas import *

matrix = []
for o in airports:
    ll = []
    for d in airports:
        dl = orig_dest_delay[(orig_dest_delay.Origin == o[0]) & (orig_dest_delay.Dest == d[0])]['delays']
        if dl.any():
            ll = ll + [dl.mean()]
        else:
            ll = ll + [0]
    matrix = matrix + [ll]
    
DataFrame(matrix)    

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,0.000000,0.000000,0.000000,0.000000,100.976568,73.314360,74.923381,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
1,0.000000,0.000000,0.000000,0.000000,85.008041,51.962465,65.027388,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
2,0.000000,0.000000,0.000000,0.000000,0.000000,35.371245,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
4,109.323704,38.912276,0.000000,0.000000,0.000000,72.791585,147.333333,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
5,65.933374,67.827945,74.024908,0.000000,82.910477,0.000000,64.436456,54.481939,48.283333,58.309085,...,0,0,0,0,0,0.000000,0.000000,0,0,0
6,76.067461,53.691641,0.000000,0.000000,46.500000,66.314946,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
7,0.000000,0.000000,0.000000,0.000000,0.000000,64.282808,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
8,0.000000,0.000000,0.000000,0.000000,0.000000,62.308199,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0
9,0.000000,0.000000,0.000000,0.000000,0.000000,66.319499,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0.000000,0.000000,0,0,0


In [104]:
from sklearn import preprocessing

#min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))

m = matrix 
#m = min_max_scaler.fit_transform(m)

m = map(lambda l: map(lambda x: int(x), l), list(m))
m

[[0,
  0,
  0,
  0,
  100,
  73,
  74,
  0,
  0,
  0,
  123,
  80,
  83,
  93,
  91,
  74,
  134,
  0,
  0,
  119,
  73,
  82,
  129,
  76,
  115,
  107,
  88,
  0,
  24,
  85,
  173,
  0,
  69,
  74,
  93,
  56,
  82,
  0,
  0,
  75,
  0,
  63,
  76,
  0,
  0,
  133,
  94,
  156,
  98,
  0,
  111,
  122,
  0,
  106,
  0,
  0,
  0,
  65,
  86,
  0,
  75,
  0,
  0,
  0,
  84,
  98,
  81,
  133,
  105,
  87,
  0,
  0,
  106,
  0,
  87,
  0,
  0,
  0,
  0,
  0,
  80,
  0,
  0,
  0,
  0,
  109,
  410,
  95,
  0,
  0,
  53,
  0,
  0,
  0,
  0,
  91,
  59,
  0,
  89,
  116,
  0,
  102,
  0,
  0,
  96,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  116,
  0,
  86,
  79,
  0,
  0,
  109,
  0,
  113,
  0,
  105,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  111,
  0,
  0,
  0,
  0,
  95,
  0,
  0,
  0,
  110,
  0,
  0,
  0,
  152,
  0,
  0,
  0,
  0,
  74,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [105]:
with open("matrix.json", "w") as fout:
    fout.write(pprint.pformat(m))